In [1]:
import tensorflow as tf
import keras
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
import os
import cv2
import glob
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from keras.layers import (Input, Activation, Conv3D, Dense, Dropout, Flatten,
                          MaxPooling3D, Input, average)
from keras.models import Model
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

In [3]:
from sklearn.model_selection import train_test_split

# All hyperparameter and constants

In [4]:
batch = 32
epoch = 10

videos = '/content/drive/My Drive/KTH_dataset'
nclass = 6
output ='/content/drive/My Drive/activity_output'

color = False
skip = True
depth = 32
nmodel = 2

if not os.path.isdir(output):
    os.makedirs(output)

img_rows, img_cols, frames=120, 120, depth
channel=3 if color else 1

nb_classes = nclass
countlis = 0

# Loading pickle Data from local directory

In [5]:
import pickle
with open('Dataset_activity/data', 'rb') as f:
    Xin = pickle.load(f)
print(Xin.shape)
with open('Dataset_activity/label', 'rb') as f:
    Yin = pickle.load(f)
print(Yin.shape)


(598, 32, 120, 120)
(598, 6)


# Splitting data for training and validation

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(Xin, Yin, test_size=0.3, random_state=4)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(418, 32, 120, 120) (418, 6)
(180, 32, 120, 120) (180, 6)


## Expanding data to a 5th axis for compatibity

In [7]:
X_train = np.expand_dims(X_train,4)
X_test = np.expand_dims(X_test,4)
print(X_train.shape,X_test.shape)

(418, 32, 120, 120, 1) (180, 32, 120, 120, 1)


# Creating sequential Model Structure

In [8]:
def create_3dcnn(shape, nb_classes):
    model = Sequential()
    model.add(Conv3D(16, kernel_size=(11,11,32), activation='relu', kernel_regularizer=l2(0.01),input_shape=(shape),padding="same"))
    model.add(Activation('sigmoid'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))
    #model.add(Dropout(0.25))
    #print("coming",input_shape)

    model.add(Conv3D(16, kernel_size=(5,5,32),activation='relu',kernel_regularizer=l2(0.01),padding="same"))
    #model.add(Activation('relu'))
    
    model.add(MaxPooling3D(pool_size=(2,2,2)))
    
    
    #model.add(Dropout(0.25))

    model.add(Flatten())
    #model.add(Dense(6400, activation='sigmoid'))
    #model.add(Dropout(0.25))
    model.add(Dense(nb_classes, activation='softmax', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))

    return model

## Validating Model

In [9]:
shape = (32,120,120,1)  # creating shape compatible with 
model = create_3dcnn((shape), nb_classes)
for layer in model.layers:
    print(layer.get_output_at(0).get_shape().as_list())
early_stopping = EarlyStopping()

[None, 32, 120, 120, 16]
[None, 32, 120, 120, 16]
[None, 16, 60, 60, 16]
[None, 16, 60, 60, 16]
[None, 8, 30, 30, 16]
[None, None]
[None, 6]


# Getting Summary of the Model parameters

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 32, 120, 120, 16)  61968     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 16, 60, 60, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 16, 60, 60, 16)    204816    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 8, 30, 30, 16)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 115200)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                

## Importing Optimizers

In [11]:
from keras.optimizers import SGD
opt = SGD(lr=0.0001) #change learning rates according to user need.

# Compile and fit Model

In [12]:
models=[]
nmodel = 1


for i in range(nmodel):
    print('model{}:'.format(i))
    models.append(model)
    models[-1].compile(loss='categorical_crossentropy',
                  optimizer=opt, metrics=['accuracy'])
    history = models[-1].fit(X_train, Y_train, validation_data=(
        X_test, Y_test), batch_size=16, epochs=800, verbose=1,shuffle=True)


model0:
Train on 418 samples, validate on 180 samples
Epoch 1/800
418/418 [==============================] - 208s 498ms/step - loss: 2.2319 - accuracy: 0.1603 - val_loss: 2.3315 - val_accuracy: 0.1722
Epoch 2/800
418/418 [==============================] - 162s 388ms/step - loss: 2.1341 - accuracy: 0.1651 - val_loss: 2.1435 - val_accuracy: 0.1444
Epoch 3/800
418/418 [==============================] - 162s 387ms/step - loss: 2.1083 - accuracy: 0.1388 - val_loss: 2.1032 - val_accuracy: 0.1556
Epoch 4/800
418/418 [==============================] - 162s 388ms/step - loss: 2.0945 - accuracy: 0.1722 - val_loss: 2.1002 - val_accuracy: 0.1722
Epoch 5/800
418/418 [==============================] - 162s 388ms/step - loss: 2.0961 - accuracy: 0.1699 - val_loss: 2.0949 - val_accuracy: 0.1722
Epoch 6/800
418/418 [==============================] - 162s 388ms/step - loss: 2.0922 - accuracy: 0.1507 - val_loss: 2.1005 - val_accuracy: 0.1500
Epoch 7/800
418/418 [==============================] - 162s 388m

418/418 [==============================] - 162s 389ms/step - loss: 2.0291 - accuracy: 0.2368 - val_loss: 2.0200 - val_accuracy: 0.3389
Epoch 57/800
418/418 [==============================] - 162s 388ms/step - loss: 2.0183 - accuracy: 0.2895 - val_loss: 2.0494 - val_accuracy: 0.1889
Epoch 58/800
418/418 [==============================] - 162s 389ms/step - loss: 2.0174 - accuracy: 0.2895 - val_loss: 2.0205 - val_accuracy: 0.2889
Epoch 59/800
418/418 [==============================] - 162s 389ms/step - loss: 2.0184 - accuracy: 0.2943 - val_loss: 2.0330 - val_accuracy: 0.1611
Epoch 60/800
418/418 [==============================] - 162s 389ms/step - loss: 2.0113 - accuracy: 0.2943 - val_loss: 2.0552 - val_accuracy: 0.1667
Epoch 61/800
418/418 [==============================] - 162s 388ms/step - loss: 2.0045 - accuracy: 0.3062 - val_loss: 2.0292 - val_accuracy: 0.2056
Epoch 62/800
418/418 [==============================] - 162s 389ms/step - loss: 2.0124 - accuracy: 0.3038 - val_loss: 2.0105 

418/418 [==============================] - 163s 390ms/step - loss: 1.5579 - accuracy: 0.4115 - val_loss: 1.8278 - val_accuracy: 0.2722
Epoch 112/800
418/418 [==============================] - 163s 390ms/step - loss: 1.5940 - accuracy: 0.3565 - val_loss: 1.9350 - val_accuracy: 0.2000
Epoch 113/800
418/418 [==============================] - 163s 391ms/step - loss: 1.5494 - accuracy: 0.3923 - val_loss: 1.5288 - val_accuracy: 0.3833
Epoch 114/800
418/418 [==============================] - 164s 391ms/step - loss: 1.5326 - accuracy: 0.4043 - val_loss: 1.5267 - val_accuracy: 0.4056
Epoch 115/800
418/418 [==============================] - 163s 390ms/step - loss: 1.5356 - accuracy: 0.3517 - val_loss: 1.5368 - val_accuracy: 0.3833
Epoch 116/800
418/418 [==============================] - 163s 389ms/step - loss: 1.5325 - accuracy: 0.3828 - val_loss: 1.5302 - val_accuracy: 0.4000
Epoch 117/800
418/418 [==============================] - 163s 390ms/step - loss: 1.5061 - accuracy: 0.4115 - val_loss: 2

ResourceExhaustedError:  OOM when allocating tensor with shape[16,16,32,120,120] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradients/max_pooling3d_1/MaxPool3D_grad/MaxPool3DGrad (defined at c:\users\bipla\anaconda3\envs\apptest\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_788]

Function call stack:
keras_scratch_graph


## Evaluate Model

In [1]:
model_inputs = [Input(shape=shape) for _ in range (nmodel)]
model_outputs = [models[i](model_inputs[i]) for i in range (nmodel)]

model = Model(inputs=model_inputs, outputs=model_outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

plot_model(model, show_shapes=True,
      to_file=os.path.join(output, 'model.png'))

model.save_weights(os.path.join(output, 'KTH_3_class_new.hd5'))

loss, acc=model.evaluate([X_test]*nmodel, Y_test, verbose=0)
with open(os.path.join(output, 'result.txt'), 'w') as f:
    f.write('Test loss: {}\nTest accuracy:{}'.format(loss, acc))

print('merged model:')
print('Test loss:', loss)
print('Test accuracy:', acc)

NameError: name 'nmodel' is not defined